In [ ]:
import numpy as np
from scipy.optimize import minimize

In [ ]:
l = 10000
data = []
for i in range(l):
    p = [1/7,3/7,1/7,1/14,1/14,1/14,1/14]
    indexes = [0,1,2,3,4,5,6]
    tep = [0,0,0,0,0,0,0]
    ind_choiced = np.random.choice(indexes,p=p)
    tep[ind_choiced] = 1
    data.append(tep.copy())
    tep[ind_choiced] = 0
data = np.array(data)

In [ ]:
def wc(ln):
    # 给定一个参考信息粒ln，求对应的wc向量
    
    T = len(ln)
    wc = np.zeros((T))
    index = np.where(ln == 1)[0][0]
    
    for i in range(index, -1, -1):
        wc[i] = 1 - (index - i) * 1 / (T - 1)
        
    for j in range(index+1, T):
        wc[j] = 1 - (j - index) * 1 / (T - 1)
    return wc

In [ ]:
def coverage_new(A,p):
    dis  = np.linalg.norm(A-p)
    dis_normalized = dis / np.sqrt(len(A))
    return 1-dis_normalized

In [ ]:
def coverage(A, ln):
    # 给定一个order-2信息粒A和一个参考信息粒ln，求A对ln的覆盖性
    w = wc(ln)
    return np.dot(A, w)

In [ ]:
def avg_coverage(A, data):
    sum_coverage = 0
    for i in range(len(data)):
        sum_coverage += coverage(A, data[i])
    return sum_coverage / len(data)

In [ ]:
def ws(A):
    # 给定一个order-2信息粒，求对应的ws向量
    
    T = len(A)
    ws = np.zeros((T))
    
    indexes = np.where(A == np.max(A))[0]
    l = len(indexes)
    
    if l % 2 != 0:
        target_index = indexes[int((l - 1) / 2)]
        for i in range(target_index, -1, -1):
            ws[i] = 1 - (target_index - i) * 1 / (T - 1)
        
        for j in range(target_index+1, T):
            ws[j] = 1 - (j - target_index) * 1 / (T - 1)
    
    else:
        target_index_1 = indexes[int(l / 2)]
        target_index_2 = indexes[int(l / 2) - 1]
        
        for i in range(target_index_1, -1, -1):
            ws[i] = 1 - (target_index_1 - i) * 1 / (T - 1)
        
        for j in range(target_index_1+1, T):
            ws[j] = 1 - (j - target_index_1) * 1 / (T - 1)
        
        ws1 = ws.copy()
        specificity_1 = np.dot(A, ws1)
        
        for i in range(target_index_2, -1, -1):
            ws[i] = 1 - (target_index_2 - i) * 1 / (T - 1)
        
        for j in range(target_index_2+1, T):
            ws[j] = 1 - (j - target_index_2) * 1 / (T - 1)
        
        ws2 = ws.copy()
        specificity_2 = np.dot(A, ws2)

        if specificity_1 > specificity_2:
            ws = ws1
    return ws

In [ ]:
def specificity(A):
    T = len(A)
    m = np.max(A)
    if np.sum(A == m) == T:
        return 0
    else:
        w = ws(A)
        return (np.dot(A, w) - 0.5) / 0.5

In [ ]:
# e = 1e-10 # 非常接近0的值
# a = 0.5
# b = 0.5
# fun = lambda A : -(a*avg_coverage(A, data) + b*specificity(A)) # 约束函数

# cons = ({'type': 'eq', 'fun': lambda A: np.sum(A) - 1}, # xyz=1
#         {'type': 'ineq', 'fun': lambda A: A[0] - e}, 
#         {'type': 'ineq', 'fun': lambda A: A[1] - e},
#         {'type': 'ineq', 'fun': lambda A: A[2] - e},
#         {'type': 'ineq', 'fun': lambda A: A[3] - e}, 
#         {'type': 'ineq', 'fun': lambda A: A[4] - e},
#         {'type': 'ineq', 'fun': lambda A: A[5] - e},
#         {'type': 'ineq', 'fun': lambda A: A[6] - e}
#        )
# A0 = np.array((0, 1, 0, 0, 0, 0, 0)) # 设置初始值
# res = minimize(fun, x0, method='SLSQP', constraints=cons)
# print('最小值：',res.fun)
# print('最优解：',res.x)
# print('迭代终止是否成功：', res.success)
# print('迭代终止原因：', res.message)


In [ ]:
bounds = [(0,1), (0,1), (0,1), (0,1), (0,1), (0,1), (0,1)]

a = 0.5
b = 0.5
p = np.array([1/7,3/7,1/7,1/14,1/14,1/14,1/14])

class PSO():
    def __init__(self, pN, dim, max_iter, c1, c2, c3, k, epsilon0):

        self.k = k  ## 近邻个数

        self.c1 = c1  # 加速因子
        self.c2 = c2  # 加速因子
        self.c3 = c3  # 加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数

        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.Xfv = np.full((self.pN, 2), 1.0e78)

        self.gg = np.zeros((self.dim))
        self.gg_fit = np.full((2), 1.0e78)  # 全局最佳适应值
        
        self.O = np.zeros((max_iter))
        self.coverage = np.zeros((max_iter))
        self.specificity = np.zeros((max_iter))

        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full((self.pN, 2), 1.0e78)  # 每个个体的历史最佳适应值

        self.epsilon = epsilon0
        self.epsilon0 = epsilon0
    
    def f(self, A):
        obj = -(coverage_new(A, p)*specificity(A))
        h = abs(np.sum(A) - 1)
        return obj, h

    def fit(self, f1, phi1, f2, phi2, epsilon):
        """
        epsilon约束处理技术
        :param f: 目标函数
        :param X1: 第一个待对比的解
        :param X2: 第二个待对比的解
        :param epsilon: 参数
        :return: 按照epsilon约束处理规则，如果X1优于X2，返回Ture，否则False
        """
        flag = False

        if phi1 <= epsilon and phi2 <= epsilon and f1 < f2:
            flag = True
        if phi1 == phi2 and f1 < f2:
            flag = True
        if phi1 < phi2 and (phi1 > epsilon or phi2 > epsilon):
            flag = True

        return flag 

    def init_Population(self):
        """
        初始化种群
        """
        for i in range(self.pN):
            for j in range(self.dim):
                self.gg[j] = self.pbest[i][j] = self.X[i][j] = bounds[j][0] \
                                                                  + (bounds[j][1] - bounds[j][0]) * np.random.uniform(0,1)
        """
        如果初始epsilon不等于0，初始化epsilon
        """
        if self.epsilon0 != 0:
            phix = []
            for i in range(self.pN):
                res = np.array(self.f(self.X[i])[1::])
                phix.append(np.sum(res[res > 0]))
            phix = sorted(phix)
            self.epsilon0 = phix[int(0.2 * self.pN)]  ###top0.2

        """
        初始化（目标函数，约束违反量）
        """
        for i in range(self.pN):
            res = np.array(self.f(self.X[i]))
            self.Xfv[i][0] = self.p_fit[i][0] = res[0]
            v = res[1::]
            phi = np.sum(v[v > 0])
            self.Xfv[i][1] = self.p_fit[i][1] = phi
            if self.fit(res[0], phi, self.gg_fit[0], self.gg_fit[1], self.epsilon0):
                self.gg = self.X[i].copy()
                self.gg_fit[0] = res[0]
                self.gg_fit[1] = phi

    def iterator(self):
        
        """
        迭代
        """
        for t in range(self.max_iter):
            curreSet = {i for i in range(self.pN)}
            if t < 0.2*self.max_iter:
                self.epsilon = self.epsilon0 * (1 - t / (0.2*self.max_iter))**5
            else:
                self.epsilon = 0

            """
            划分第一个子种群并进化
            """
            indexes = np.full(self.k, 0)
            Uk = np.zeros((self.dim))
            for j in range(self.dim):
                Uk[j] = bounds[j][0] + (bounds[j][1] - bounds[j][0]) \
                        * np.random.uniform(0,1)
            mindistan = 1.0e+70
            minindex = 0

            for i in curreSet:
                distan = np.linalg.norm(self.X[i] - Uk)
                if distan < mindistan:
                    mindistan = distan
                    minindex = i
            curreSet = curreSet - {minindex}
            indexes[0] = minindex

            for j in range(1, self.k):
                mindistan = 1.0e+70
                minindex = 0
                for i in curreSet:
                    distan = np.linalg.norm(self.X[i] - self.pbest[indexes[0]])
                    if distan < mindistan:
                        mindistan = distan
                        minindex = i
                curreSet = curreSet - {minindex}
                indexes[j] = minindex

            """
            进化第一个种群
            """
            for j in range(self.k):
                insets = set(indexes) - {indexes[j]}

                a = np.random.choice(list(insets))
                insets = insets - {a}
                b = np.random.choice(list(insets))
                self.X[indexes[j]] = self.c1 * np.random.uniform(0,1) * self.pbest[indexes[j]] \
                                     + self.c2 * np.random.uniform(0,1) * self.gg \
                                     + self.c3 * np.random.uniform(0,1) * (self.pbest[a] - self.pbest[b])
                """
                控制在搜索范围内
                """
                for d in range(self.dim):
                    if self.X[indexes[j]][d] < bounds[d][0]:
                        self.X[indexes[j]][d] = bounds[d][0]
                    if self.X[indexes[j]][d] > bounds[d][1]:
                        self.X[indexes[j]][d] = bounds[d][1]

                res = np.array(self.f(self.X[indexes[j]]))
                v = res[1::]
                phi = np.sum(v[v > 0])

                if self.fit(res[0], phi, self.p_fit[indexes[j]][0], self.p_fit[indexes[j]][1], self.epsilon):
                    self.p_fit[indexes[j]][0] = res[0]
                    self.p_fit[indexes[j]][1] = phi
                    self.pbest[indexes[j]] = self.X[indexes[j]].copy()
                    if self.fit(res[0], phi, self.gg_fit[0], self.gg_fit[1], self.epsilon):
                        self.gg_fit[0] = res[0]
                        self.gg_fit[1] = phi
                        self.gg = self.X[indexes[j]].copy()


                print((self.gg_fit[0], self.gg_fit[1], t * 201 + (j+1)))

            """
            划分其他子种群并进化
            """
            for n in range(1, int(self.pN / self.k)):
                preindexes = indexes
                indexes = np.full(self.k, 0)
                Uk = np.zeros((self.dim))
                for j in range(self.dim):
                    Uk[j] = bounds[j][0] + (bounds[j][1] - bounds[j][0])\
                                   * np.random.uniform(0,1)
                mindistan = 1.0e+70
                minindex = 0

                for i in curreSet:
                    distan = np.linalg.norm(self.X[i] - Uk)
                    if distan < mindistan:
                        mindistan = distan
                        minindex = i
                curreSet = curreSet - {minindex}
                indexes[0] = minindex

                for j in range(1,self.k):
                    mindistan = 1.0e+70
                    minindex = 0
                    for i in curreSet:
                        distan = np.linalg.norm(self.X[i] - self.pbest[indexes[0]])
                        if distan < mindistan:
                            mindistan = distan
                            minindex = i
                    curreSet = curreSet - {minindex}
                    indexes[j] = minindex

                for j in range(self.k):
                    insets = set(indexes) - {indexes[j]}
                    c = np.random.choice(list(insets))

                    self.X[indexes[j]] = self.c1 * np.random.uniform(0,1) * self.pbest[indexes[j]] \
                                         + self.c2 * np.random.uniform(0,1) * self.gg \
                                         + self.c3 * np.random.uniform(0,1) * (self.pbest[preindexes[j]] - self.pbest[c])
                    """
                    控制在搜索范围内
                    """
                    for d in range(self.dim):
                        if self.X[indexes[j]][d] < bounds[d][0]:
                            self.X[indexes[j]][d] = bounds[d][0]
                        if self.X[indexes[j]][d] > bounds[d][1]:
                            self.X[indexes[j]][d] = bounds[d][1]

                    res = np.array(self.f(self.X[indexes[j]]))
                    v = res[1::]
                    phi = np.sum(v[v > 0])

                    if self.fit(res[0], phi, self.p_fit[indexes[j]][0], self.p_fit[indexes[j]][1], self.epsilon):
                        self.p_fit[indexes[j]][0] = res[0]
                        self.p_fit[indexes[j]][1] = phi
                        self.pbest[indexes[j]] = self.X[indexes[j]].copy()
                        if self.fit(res[0], phi, self.gg_fit[0], self.gg_fit[1], self.epsilon):
                            self.gg_fit[0] = res[0]
                            self.gg_fit[1] = phi
                            self.gg = self.X[indexes[j]].copy()


                    print((self.gg_fit[0], self.gg_fit[1], t * 201 + 10*n + (j+1)))

            """
            记忆种群进化
            """
            for i in range(self.pN):
                indexes = {j for j in range(self.pN)} - {i}
                e = np.random.choice(list(indexes))
                pbest_new = self.pbest[i].copy()
                if self.fit(self.p_fit[i][0], self.p_fit[i][1], self.p_fit[e][0], self.p_fit[e][1], self.epsilon):
                    f = np.random.choice(list(indexes))
                    indexes = indexes - {f}
                    g = np.random.choice(list(indexes))
                    indexes = indexes - {g}
                    h = np.random.choice(list(indexes))
                    for j in range(self.dim):
                        if np.random.uniform(0, 1) > 0.5:
                            pbest_new[j] = self.pbest[f][j] \
                                               + (np.random.uniform(-1, 1)) * (self.pbest[g][j] - self.pbest[h][j])
                else:
                    l = np.random.choice(list(indexes))
                    indexes = indexes - {l}
                    m = np.random.choice(list(indexes))
                    pbest_new = self.pbest[i] + np.random.uniform(0,1) * (self.pbest[e] - self.pbest[i]) \
                                    + np.random.uniform(0,1) * (self.pbest[l] - self.pbest[m])

                """
                控制在搜索范围内
                """
                for d in range(self.dim):
                    if pbest_new[d] < bounds[d][0]:
                        pbest_new[d] = bounds[d][0]
                    if pbest_new[d] > bounds[d][1]:
                        pbest_new[d] = bounds[d][1]

                res = np.array(self.f(pbest_new))
                v = res[1::]
                phi = np.sum(v[v > 0])

                if self.fit(res[0], phi, self.p_fit[i][0], self.p_fit[i][1], self.epsilon):
                    self.p_fit[i][0] = res[0]
                    self.p_fit[i][1] = phi
                    self.pbest[i] = pbest_new.copy()
                    if self.fit(res[0], phi, self.gg_fit[0], self.gg_fit[1], self.epsilon):
                        self.gg_fit[0] = res[0]
                        self.gg_fit[1] = phi
                        self.gg = pbest_new.copy()

                print((self.gg_fit[0], self.gg_fit[1], t * 201 + 100 + (i+1)))

            """
            全局最优解的混合
            """
            gg_new = self.gg.copy()
            if np.random.uniform(0,1) < (1 / self.dim):
                dr = np.random.choice([i for i in range(self.dim)])
                gg_new[dr] = self.gg[dr] + (np.random.uniform(0,1)) * (bounds[dr][1] - bounds[dr][0])
                if gg_new[dr] < bounds[dr][0]:
                    gg_new[dr] = bounds[dr][0]
                if gg_new[dr] > bounds[dr][1]:
                    gg_new[dr] = bounds[dr][1]

            res = np.array(self.f(gg_new))
            v = res[1::]
            phi = np.sum(v[v > 0])

            if self.fit(res[0], phi, self.gg_fit[0], self.gg_fit[1], self.epsilon):
                self.gg_fit[0] = res[0]
                self.gg_fit[1] = phi
                self.gg = gg_new.copy()

            self.coverage[t] = coverage_new(self.gg, p)
            self.specificity[t] = specificity(self.gg)
            self.O[t] = self.coverage[t] * self.specificity[t]
            print((self.gg_fit[0], self.gg_fit[1], t * 201 + 201))


        return 1


In [ ]:
# P = CMPSOWV.PSO(100, G01.dim, 2488, 4.1/3, 4.1/3, 4.1/3, 10, 0)
# def __init__(self, pN, dim, max_iter, c1, c2, c3, k, epsilon0):
my_pso = PSO(100, 7, 1000, 4.1/3, 4.1/3, 4.1/3, 10, 1)
my_pso.init_Population()
my_pso.iterator()

In [ ]:
my_pso.gg

In [ ]:
coverage_new(my_pso.gg, p)

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
##### 定义图片上的字体
font = {'family': 'Times New Roman',
         'style': 'normal',
         'weight': 'light',
        #'color':  'darkred', 
         'size': 10,
        }
matplotlib.rc('font', **font)

In [ ]:
n_groups = 7
 
fig, ax = plt.subplots()
 
index = np.arange(n_groups)
bar_width = 0.4
 
opacity = 1
error_config = {'ecolor': '0.3'}
 
rects1 = ax.bar(index, p, bar_width,
                alpha=opacity, 
                error_kw=error_config,
                label="Statistical average")
 
rects2 = ax.bar(index + bar_width, my_pso.gg, bar_width,
                alpha=opacity, 
                error_kw=error_config,
                label='PSO optimized')
 
# rects3 = ax.bar(index + bar_width + bar_width, data_23, bar_width,
#                 alpha=opacity, 
#                 error_kw=error_config,
#                 label='$M=23$')

# rects4 = ax.bar(index + bar_width * 3, data_33, bar_width,
#                 alpha=opacity, 
#                 error_kw=error_config,
#                 label='$M=33$')
 
ax.set_xticks(index + 2 * bar_width / 4)
ax.set_xticklabels(("$p_1$", "$p_2$", "$p_3$", "$p_4$", "$p_5$", "$p_6$", "$p_7$"))
ax.legend()
plt.xlabel("Membership degree")
plt.ylabel("Value(s)")
# plt.grid()
fig.tight_layout()
plt.savefig("./results/Example 3/Membership.svg", format='svg', dpi=1000)
plt.show()

In [ ]:
x = np.linspace(1,1001,1000)
plt.xlim(1,1000)
plt.ylim(0.5, 3)
plt.ylabel("Values", size=10)
plt.xlabel("Number of iterations", size=10)


# plt.plot(x, my_pso.O, ls="-.", label="Objective function value")
plt.plot(x, my_pso.coverage, ls="-",  label="Coverage")
plt.plot(x, my_pso.specificity, ls="-", label="Specificity")
plt.axhline(1, ls="--", color="r", linewidth=0.8)

plt.legend()
plt.savefig("./results/Example 3/o_cov_spe.svg", format='svg', dpi=1000)
plt.show()

In [ ]:
x = np.linspace(1,301,300)
plt.xlim(1,300)
plt.ylim(0.5, 3)
plt.ylabel("Values", size=10)
plt.xlabel("Number of iterations", size=10)


# plt.plot(x, my_pso.O, ls="-.", label="Objective function value")
plt.plot(x, my_pso.coverage[0:300], ls="-",  label="Coverage")
plt.plot(x, my_pso.specificity[0:300], ls="-", label="Specificity")
plt.axhline(1, ls="--", color="r", linewidth=0.8)

plt.legend()
plt.savefig("./results/Example 3/o_cov_spe_xiangxi.svg", format='svg', dpi=1000)
plt.show()